In [4]:
!echo "== collection =="
!head -n 2 data/collection/collection.tsv
!echo "== queries =="
!head -n 2 data/queries/queries.train.tsv
!echo "== top1000 =="
!head -n 2 data/top1000/top1000.dev.tsv

# https://microsoft.github.io/msmarco/Submission

== collection ==
0	The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
1	The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.
== queries ==
121352	define extreme
634306	what does chattel mean on credit history
== top1000 ==
188714	1000052	foods and supplements to lower blood sugar	Watch portion sizes: ■ Even healthy foods will cause high blood sugar if you eat too much. ■ Make sure each of your meals has the same amount of CHOs. Avoid foods high in sugar: ■ Some foods to avoid: sugar, honey, candies, syrup, cakes, cookies, regular soda and.
1082792	1000084	what does the golgi apparatus do to the 

In [2]:
import pandas as pd
import os

In [10]:
collection  = pd.read_csv("./data/collection/collection.tsv", header=None, sep="\t", index_col=0, names=["passage"])
query_train = pd.read_csv("./data/queries/queries.train.tsv", header=None, sep="\t", index_col=0, names=["query"])
query_eval  = pd.read_csv("./data/queries/queries.eval.tsv", header=None, sep="\t", index_col=0, names=["query"])
query_dev   = pd.read_csv("./data/queries/queries.dev.tsv", header=None, sep="\t", index_col=0, names=["query"])

In [2]:
top1000_dev = pd.read_csv("./data/top1000/top1000.dev.tsv", header=None, sep="\t", names=["qid", "pid", "query", "passage"])
print(len(top1000_dev))

6668967


In [3]:
if not os.path.exists("./data/top1000/top1000.dev.ids.tsv"):
  t = top1000_dev.drop(columns=["query", "passage"])
  t.to_csv("./data/top1000/top1000.dev.ids.tsv", header=None, sep="\t", index=None)
!echo "== top1000.ids =="
!head -n 2 data/top1000/top1000.dev.ids.tsv

== top1000.ids ==
188714	1000052
1082792	1000084


In [7]:
df2 = top1000_dev.drop(columns=["pid", "passage"]).drop_duplicates()
print(len(df2))
if not os.path.exists("./data/top1000/top1000.dev.inputs.tsv"):
  df2.to_csv("./data/top1000/top1000.dev.inputs.tsv", header=None, sep="\t", index=None)
!echo "== top1000.dev.inputs =="
!head -n 2 data/top1000/top1000.dev.inputs.tsv

6980
== top1000.dev.inputs ==
188714	foods and supplements to lower blood sugar
1082792	what does the golgi apparatus do to the proteins and lipids once they arrive ?


In [19]:
print(f"{len(collection)}")
collection

8841823


,passage
0,The presence of communication amid scientific ...
1,The Manhattan Project and its atomic bomb help...
2,Essay on The Manhattan Project - The Manhattan...
3,The Manhattan Project was the name for a proje...
4,versions of each volume as well as complementa...
...,...
8841818,When metal salts emit short wavelengths of vis...
8841819,Thousands of people across the United States w...
8841820,"The recipe that creates blue, for example, inc..."
8841821,"On Independence Days of yore, old-timey crowds..."


In [20]:
print(f"{len(query_train)=} {len(query_eval)=} {len(query_dev)=}")
query_dev

len(query_train)=808731 len(query_eval)=101092 len(query_dev)=101093


,query
1048578,cost of endless pools/swim spa
1048579,what is pcnt
1048580,what is pcb waste
1048581,what is pbis?
1048582,what is paysky
...,...
480594,"price of copper by ounce, pound"
524271,trazodone for dogs side effects
1048565,who plays sebastian michaelis
1048570,what is pearls before swine?


In [26]:
print(f"{len(top1000_dev)=}")
top1000_dev

len(top1000_dev)=6668967


,qid,pid,query,passage
0,188714,1000052,foods and supplements to lower blood sugar,Watch portion sizes: ■ Even healthy foods will...
1,1082792,1000084,what does the golgi apparatus do to the protei...,"Start studying Bonding, Carbs, Proteins, Lipid..."
2,995526,1000094,where is the federal penitentiary in ind,It takes THOUSANDS of Macy's associates to bri...
3,199776,1000115,health benefits of eating vegetarian,The good news is that you will discover what g...
4,660957,1000115,what foods are good if you have gout?,The good news is that you will discover what g...
...,...,...,...,...
6668962,679360,999933,what is a corporate bylaws,Corporate Records for Nonprofit Corporations. ...
6668963,36388,999956,average family savings account,When it comes to average retirement savings st...
6668964,43781,999956,average savings per age group,When it comes to average retirement savings st...
6668965,28442,999956,at what age does the average person retire,When it comes to average retirement savings st...


In [8]:
top1000_dev.groupby("qid").count()

,pid,query,passage
qid,,,
2,1000,1000,1000
1215,1000,1000,1000
1288,1000,1000,1000
1576,1000,1000,1000
2235,1000,1000,1000
...,...,...,...
1102335,1000,1000,1000
1102351,1000,1000,1000
1102390,1000,1000,1000


# Qrels

In [28]:
qrels = pd.read_csv("./data/qrels/qrels.dev.tsv", header=None, sep="\t", names=["qid", "iteration_num", "pid", "relevance"])
qrels_qids = qrels.drop_duplicates(subset=["qid"]).filter(items=["qid"]).reset_index(drop=True)
print(len(qrels_qids), len(qrels["qid"].unique()))
qrels_qids

55578 55578


,qid
0,1102432
1,1102431
2,1090282
3,39449
4,76162
...,...
55573,150337
55574,22241
55575,129177
55576,190655


In [17]:
print(qrels.groupby("qid").count()["relevance"].unique())
print(len(qrels["qid"].unique()))

[1 3 2 6 5 4]
55578


In [22]:
query_dev = pd.read_csv("./data/queries/queries.dev.tsv", header=None, sep="\t", index_col=0, names=["query"])
for qrel in qrels_qids:
  assert qrel in query_dev.index

In [23]:
qrels_qids.join

AttributeError: 'numpy.ndarray' object has no attribute 'join'